In [118]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

OPCIONES FORMATO

In [119]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [120]:
dtypes = {'id': 'int32', 
'titulo': 'object', 
'descripcion': 'object', 
'tipodepropiedad': 'category', 
'direccion': 'object', 
'ciudad': 'object', 
'provincia': 'category', 
'antiguedad': 'float16', 
'habitaciones': 'float16', 
'garages': 'float16', 
'banos': 'float16', 
'metroscubiertos': 'float16', 
'metrostotales': 'float16', 
'idzona': 'object', 
'lat': 'float64', 
'lng': 'float64', 
'gimnasio': 'float16', 
'usosmultiples': 'float16', 
'piscina': 'float16', 
'escuelascercanas': 'float16', 
'centroscomercialescercanos': 'float16', 
'precio': 'float32', }

data = pd.read_csv("data/train.csv", dtype = dtypes)
data.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,nan,2.00,1.00,...,23533.0,nan,nan,2015-08-23 00:00:00,0.00,0.00,0.00,0.00,0.00,2273000.00
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.00,3.00,2.00,...,24514.0,19.31,-99.23,2013-06-28 00:00:00,0.00,0.00,0.00,1.00,1.00,3600000.00
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.00,3.00,2.00,...,48551.0,nan,nan,2015-10-17 00:00:00,0.00,0.00,0.00,0.00,0.00,1200000.00
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.00,2.00,1.00,...,53666.0,19.30,-99.69,2012-03-09 00:00:00,0.00,0.00,0.00,1.00,1.00,650000.00
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.00,2.00,1.00,...,47835.0,nan,nan,2016-06-07 00:00:00,0.00,0.00,0.00,0.00,0.00,1150000.00


# Pre-procesamiento de data

In [121]:
data.isnull().sum().sum()

535127

In [122]:
data.size

5520000

In [12]:
#(amount_of_nulls/(data.size))*100

Los Nulls representan el 10% de los datos.

In [123]:
data.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

Se eliminan los Nulls hasta nuevo aviso. Pero para XGBoost no deberia ser necesario.

In [124]:
data = data.dropna()

# Preparacion del set de datos de entrenamiento (features)

La idea es preparar el set con los datos para exportar y que despues el modelo simplemente levante, separe en set de entrenamiento y test, entrene y devuelva una prediccion.

Se procede a calcular features. Cada feature se agregara al DataFrame final que tendra que levantar despues el modelo. Ojo que dentro de este DataFrame tambien va a estar el precio (que es el label).

In [125]:
train_set = pd.DataFrame()

Primero de todo, se agrega el id y el precio asocidado. A partir de esto, se agregaran los features que se consideren necesarios.

In [126]:
train_set["id"] = data["id"]
train_set["precio"] = data["precio"]

In [127]:
train_set.head()

,id,precio
1,53461,3600000.00
3,209067,650000.00
5,126147,1100000.00
7,5013,4200000.00
12,47890,1500000.00


A partir de esto se generan features.

## Se generan features para el training set

Tomando como base la información obtenida de la exploración de datos armamos features

1) Relación entre precio vs metros cuadrados de la propiedad. La idea es agregar un peso a la propiedad en función de esta relación. El precio de la propiedad puede dividirse en 2 grupos: los menores a 150 mts totales y los mayores a 150 mts totales. Los primeros se ve que tienen precios dispares dentro del rango 0~150 mts totales ; mientras que los segundos tienen un precio mucho más constante.
    * <= 150 mts totales: precio promedio * 0,8
    * >= 150 mts totales: precio promedio * 0,4

In [128]:
data.transpose()

,1,3,5,7,12,16,31,44,45,53,...,239972,239979,239981,239984,239989,239991,239993,239994,239996,239998
id,53461,209067,126147,5013,47890,283945,118889,92610,274931,198249,...,130523,103355,199041,262913,175203,137337,207892,110268,259178,146867
titulo,condominio horizontal en venta,casa sola en toluca zinacantepec con credito i...,departamento en venta taxqueña,casa emilia en venta en selvamar playa del carmen,hermosa casa en zibata queretaro 170m²,preciosa casa en cumbres del lago \t,vendo hermosa casa en fraccionamiento lomas de...,"casa en venta en agrícola oriental, iztacalco",casa en condominio en venta en jardines de vil...,departamento calle 6 agricola pantitlan q,...,"casa para remodelar, excelente ubicación",se vende casa en delegacion miguel hidalgo,casa en venta en residencial agua caliente,en venta mitras centro monterrey nuevo leon,casa venta renta en rinconada jurica,departamento residencial coyuya,bugambilias (ciudad),hermosa casa en villa de los belenes,casa en condominio a 10 min. del centro de toluca,casa sola. javier rojo gomez.
descripcion,"<p>entre sonora y guerrero, atr&aacute;s del h...",casa en privada con caseta de vigilancia casas...,"amplio departamento, estancia de sala y comedo...",casa emilia en venta playa del carmenfracciona...,<p><strong>casa </strong>en venta en fracciona...,<p>clave: vcln2450 fecha de actualizaci&oacute...,<p>vendo hermosa casa en fraccionamiento lomas...,<p>casa duplex (el primer nivel es el que se e...,-- cv2096e -- casas residenciales de tipo m...,"proyecto de departamentos de hasta 53 m2, en l...",...,"estado de conservación: para remodelar, área d...",excelente casa muy bien ubicada en zona tranqu...,se vende bonita casa en el fracc. residencial ...,en excelente ubicación zona mitras centro. am...,terreno 404m2 \nconst. 347 m2 \n\npreciosa cas...,"departamento ubicado en planta baja, con excel...",coto privado de tan solo 7 casas donde cada fa...,"<p>moderna casa 3 pisos, muro llor&oacute;n , ...","casa con un jardin amplio, un cuarto de servic...","casa sola, dividida en cuatro departamentos de..."
tipodepropiedad,Casa en condominio,Casa,Apartamento,Casa,Casa,Casa,Casa,Casa,Casa en condominio,Apartamento,...,Casa,Casa,Casa,Casa,Casa,Apartamento,Casa,Casa,Casa,Casa
direccion,AV. MEXICO,IGNACIO MANUEL ALTAMIRANO 128,Condominio Tlalpan 2B,condominio el trebol,"Pitahayas 2 - 58, Zibata","CUMBRES DEL LAGO JURIQUILLA, QUERETARO, MEXICO",FRACCIONAMIENTO LOMAS DE SAN ALFONSO,Sur 16,RESIDENCIAL D´LAGO S/N,CALLE 6 NUMERO 106,...,Albino Garcia 219,Rio Parana,PRIV. VENECIA,"PROGRESO, MONTERREY",Jurica,Coyuya 200,Paseo de la Cañada,"MANUEL AMAYA,ENTRE SEBASTIAN ALLENDE Y AMADO A...",Filiberto Navas 325,Javier Rojo Gomez 120
ciudad,La Magdalena Contreras,Zinacantepec,Coyoacán,Playa del Carmen,Querétaro,Querétaro,Puebla,Iztacalco,Villahermosa,Iztacalco,...,Cuauhtémoc,Miguel Hidalgo,Tijuana,Monterrey,Querétaro,Iztacalco,Zapopan,Zapopan,Toluca,Iztapalapa
provincia,Distrito Federal,Edo. de México,Distrito Federal,Quintana Roo,Querétaro,Querétaro,Puebla,Distrito Federal,Tabasco,Distrito Federal,...,Distrito Federal,Distrito Federal,Baja California Norte,Nuevo León,Querétaro,Distrito Federal,Jalisco,Jalisco,Edo. de México,Distrito Federal
antiguedad,10.00,1.00,5.00,2.00,0.00,0.00,5.00,16.00,5.00,0.00,...,20.00,10.00,10.00,20.00,2.00,20.00,1.00,3.00,0.00,20.00
habitaciones,3.00,2.00,2.00,4.00,3.00,3.00,3.00,3.00,3.00,2.00,...,5.00,3.00,2.00,2.00,3.00,2.00,3.00,2.00,3.00,4.00
garages,2.00,1.00,1.00,2.00,2.00,2.00,2.00,1.00,2.00,1.00,...,2.00,2.00,1.00,1.00,2.00,1.00,2.00,2.00,3.00,0.00


In [129]:
# ===== CONFIG =====
limite_metrostotales = 150
precio_dolar = 0.052
precio_feature = 'precioUS'
factor_myr = 0.8
factor_mnr = 0.4
# ====================
feature1 = data[['id','precio','metrostotales']]
#Seteo el precio en dolares(US$)
feature1['precioUS'] = feature1['precio'] * precio_dolar

feature1['pmyr'] = feature1[feature1['metrostotales'] >= limite_metrostotales][precio_feature]
feature1['pmnr'] = feature1[feature1['metrostotales'] < limite_metrostotales][precio_feature]
#Seteo los Nan en 0
feature1.fillna(0,inplace=True)
feature1.head()

/home/lucas/.local/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/lucas/.local/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/lucas/.local/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

,id,precio,metrostotales,precioUS,pmyr,pmnr
1,53461,3600000.00,180.00,187200.00,187200.00,0.00
3,209067,650000.00,67.00,33800.00,0.00,33800.00
5,126147,1100000.00,90.00,57200.00,0.00,57200.00
7,5013,4200000.00,293.00,218400.00,218400.00,0.00
12,47890,1500000.00,141.00,78000.00,0.00,78000.00


In [130]:
mym = feature1.describe()['pmyr']['mean']
mnm = feature1.describe()['pmnr']['mean']
print("Promedio mayor a limite: "+str(mym))
print("Factor mayor: "+str(mym*factor_myr))
print("Promedio menor a limite: "+str(mnm))
print("Factor menor: "+str(mnm*factor_mnr))

def factor_pm(row):
    if(row['metrostotales'] >= limite_metrostotales):
        return mym * factor_myr
    return mnm * factor_mnr

feature1['factor_pm'] = feature1.apply(f2,axis=1)
feature1.head()

Promedio mayor a limite: 70412.2734375
Factor mayor: 56329.818750000006
Promedio menor a limite: 44202.80859375
Factor menor: 17681.123437500002


/home/lucas/.local/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,id,precio,metrostotales,precioUS,pmyr,pmnr,factor_pm
1,53461,3600000.00,180.00,187200.00,187200.00,0.00,56329.82
3,209067,650000.00,67.00,33800.00,0.00,33800.00,17681.12
5,126147,1100000.00,90.00,57200.00,0.00,57200.00,17681.12
7,5013,4200000.00,293.00,218400.00,218400.00,0.00,56329.82
12,47890,1500000.00,141.00,78000.00,0.00,78000.00,17681.12


## Merge al train_set

In [131]:
feature1_to_merge = feature1[['id','factor_pm']]
train_set = train_set.merge(feature1_to_merge,on = 'id', how='inner')
train_set.head()

,id,precio,factor_pm
0,53461,3600000.00,56329.82
1,209067,650000.00,17681.12
2,126147,1100000.00,17681.12
3,5013,4200000.00,56329.82
4,47890,1500000.00,17681.12


## Primero se agregan todas las columnas numericas que ya se tienen

In [132]:
data.dtypes

id                               int32
titulo                          object
descripcion                     object
tipodepropiedad               category
direccion                       object
ciudad                          object
provincia                     category
antiguedad                     float16
habitaciones                   float16
garages                        float16
banos                          float16
metroscubiertos                float16
metrostotales                  float16
idzona                          object
lat                            float64
lng                            float64
fecha                           object
gimnasio                       float16
usosmultiples                  float16
piscina                        float16
escuelascercanas               float16
centroscomercialescercanos     float16
precio                         float32
dtype: object

In [133]:
data_num = data.loc[:,["id", "antiguedad", "habitaciones", "garages", 
                       "banos", "metroscubiertos", "lat", "lng", 
                       "usosmultiples", "piscina", "escuelascercanas",
                      "centroscomercialescercanos"]]

In [134]:
train_set = train_set.merge(data_num, on = ["id"], how = "inner")
train_set.head()

,id,precio,factor_pm,antiguedad,habitaciones,garages,banos,metroscubiertos,lat,lng,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,53461,3600000.00,56329.82,10.00,3.00,2.00,2.00,268.00,19.31,-99.23,0.00,0.00,1.00,1.00
1,209067,650000.00,17681.12,1.00,2.00,1.00,1.00,63.00,19.30,-99.69,0.00,0.00,1.00,1.00
2,126147,1100000.00,17681.12,5.00,2.00,1.00,1.00,75.00,19.30,-99.15,0.00,0.00,0.00,1.00
3,5013,4200000.00,56329.82,2.00,4.00,2.00,3.00,293.00,20.67,-87.04,0.00,0.00,0.00,0.00
4,47890,1500000.00,17681.12,0.00,3.00,2.00,2.00,170.00,20.68,-100.32,1.00,0.00,1.00,1.00


# Se exporta el train_set final para uso de los modelos

In [135]:
train_set.to_csv("train_set.csv")